https://zhuanlan.zhihu.com/p/104301061?utm_source=qq&utm_medium=social&utm_oi=50043553841152

# 计时与性能分析

In [2]:
# 下列程序用作演示目的，该程序计算 e 的 X 次方（摘自 Python 文档）：

# slow_program.py
from decimal import *

def exp(x):
    getcontext().prec += 2
    i, lasts, s, fact, num = 0, 0, 1, 1, 1
    while s != lasts:
        lasts = s
        i += 1
        fact *= i
        num *= x
        s += num / fact
    getcontext().prec -= 2
    return +s

exp(Decimal(150))
exp(Decimal(400))
exp(Decimal(3000))

Decimal('7.646200989054704889310727660E+1302')

最懒惰的「性能分析」

首先，最简单但说实话也很懒的方法——使用 Unix 的 time 命令

# 计时专用函数

现在我们知道了需要关注哪里，那么我们可能只想要给运行缓慢的函数计时而不去管代码的其他部分。我们可以使用一个简单的装饰器来做到这点：

In [12]:
from functools import wraps
import time

def timeit_wrapper(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()  # Alternatively, you can use time.process_time()
        func_return_val = func(*args, **kwargs)
        end = time.perf_counter()
        print('{0:<10}.{1:<8} : {2:<8}'.format(func.__module__, func.__name__, end - start))
        return func_return_val
    return wrapper

接着，将该装饰器按如下方式应用在待测函数上：

In [13]:
@timeit_wrapper
def exp(x):
    getcontext().prec += 2
    i, lasts, s, fact, num = 0, 0, 1, 1, 1
    while s != lasts:
        lasts = s
        i += 1
        fact *= i
        num *= x
        s += num / fact
    getcontext().prec -= 2
    return +s

print('{0:<10} {1:<8} {2:^8}'.format('module', 'function', 'time'))
exp(Decimal(150))
exp(Decimal(400))
exp(Decimal(3000))

module     function   time  
__main__  .exp      : 0.002632599999969898
__main__  .exp      : 0.03045260000004646
__main__  .exp      : 8.883606200000031


Decimal('7.646200989054704889310727660E+1302')

此时我们需要考虑想要测量哪一类时间。time 库提供了 time.perf_counter 和 time.process_time 两种时间。其区别在于，perf_counter 返回绝对值，其中包括了 Python 程序并不在运行的时间，因此它可能受到机器负载的影响。而 process_time 只返回用户时间（除去了系统时间），也就是只有进程运行时间。

# 让程序更快

现在到了真正有趣的部分了，让 Python 程序跑得更快！我不会告诉你一些奇技淫巧或代码段来神奇地解决程序的性能问题，而更多是关于通用的想法和策略。使用这些策略，可以对程序性能产生巨大的影响，有时甚至可以带来高达 30% 的提速。


## 1. 使用内置的数据类型

这一点非常明显。内置的数据类型非常快，尤其相比于树或链表等自定义类型而言。这主要是因为内置数据类型使用 C 语言实现，使用 Python 实现的代码在运行速度上和它们没法比。

## 2. 使用局部变量

这和每个作用域中变量的查找速度有关。我之所以说「每个作用域」，是因为这不仅仅关乎局部变量或全局变量。事实上，就连函数中的局部变量、类级别的属性和全局导入函数这三者的查找速度都会有区别。函数中的局部变量最快，类级别属性（如 self.name）慢一些，全局导入函数（如 time.time）最慢。

## 3. 使用函数

这也许有些反直觉，因为调用函数会让更多的东西入栈，进而在函数返回时为程序带来负担，但这其实和之前的策略相关。如果你只是把所有代码扔进一个文件而没有把它们放进函数，那么它会因为众多的全局变量而变慢。因此，你可以通过将所有代码封装在 main 函数中并调用它来实现加速，如下所示：

## 4. 不要访问属性

另一个可能让程序变慢的东西是用来访问对象属性的点运算符（.）。这个运算符会引起程序使用__getattribute__进行字典查找，进而为程序带来不必要的开销。那么，我们怎么避免（或者限制）使用它呢？